# Autograd Tutorial

# Import required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

%matplotlib inline

# Automatic Differentiation with ``torch.autograd``

When training neural networks, the most frequently used algorithm is **back propagation**. In this algorithm, parameters (model weights) are adjusted according to the **gradient** of the loss function with respect to the given parameter.

To compute those gradients, PyTorch has a built-in differentiation engine called ``torch.autograd``. It supports automatic computation of gradient for any computational graph.

Consider the simplest one-layer neural network, with input ``x``, parameters ``w`` and ``b``, and some loss function. It can be defined in PyTorch in the following manner:

In [2]:
dim_outputs = 3 # Dimension of output
x = torch.ones(5)  # input tensor
y = torch.zeros(dim_outputs)  # expected output
w = torch.randn(5, dim_outputs, requires_grad = True)
b = torch.randn(dim_outputs, requires_grad = True)
z = torch.matmul(x, w) + b

print(f"Shape of input tensor is {x.shape}")
print(f"Shape of weight tensor is {w.shape}")
print(f"Shape of computed output tensor (z) is {z.shape}")
print(f"Shape of actual output tensor (y) is {y.shape}")
print()
print(f"Input tensor is {x}")
print(f"Actual output tensor is {y}")
print(f"Computed output tensor is {z}")

Shape of input tensor is torch.Size([5])
Shape of weight tensor is torch.Size([5, 3])
Shape of computed output tensor (z) is torch.Size([3])
Shape of actual output tensor (y) is torch.Size([3])

Input tensor is tensor([1., 1., 1., 1., 1.])
Actual output tensor is tensor([0., 0., 0.])
Computed output tensor is tensor([2.1721, 0.5525, 5.6832], grad_fn=<AddBackward0>)


In [3]:
# Define loss function - Binary cross-entropy
# y_act = 0; y_pred = sigmoid(z); BCE loss = ln(1 - y_pred)
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)
loss_calc = (-1 / dim_outputs) * (torch.sum(np.log(1 - torch.nn.Sigmoid()(z).detach())))
print("Simulated loss is %0.4f" %(loss))
print("Calculated loss is %0.4f" %(loss_calc))

Simulated loss is 2.9912
Calculated loss is 2.9912


# Tensors, Functions and Computational graph

The code above defines the following **computational graph**:

<table>
<tr>
  <td>
<img src="Images/comp-graph.png" alt="A tensor shape is like a vector.">
  </td>
  </tr>
</table>

In this network, ``w`` and ``b`` are **parameters**, which we need to optimize. Thus, we need to be able to compute the gradients of loss function with respect to those variables. In order to do that, we set the ``requires_grad`` property of those tensors.


**Note:** We can set the value of ``requires_grad`` when creating a tensor, or later by using `x.requires_grad_(True)` method.


A function that we apply to tensors to construct computational graph is in fact an object of class ``Function``. This object knows how to compute the function in the *forward* direction, and also how to compute its derivative during the *backward propagation* step. A reference to the backward propagation function is stored in ``grad_fn`` property of a tensor. For more details of ``Function``, refer this [link](https://pytorch.org/docs/stable/autograd.html#function)

In [4]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x000002201A9064C0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x000002201A9063D0>


# Computing Gradients

To optimize weights of parameters in the neural network, we need to compute the derivatives of our loss function with respect to parameters, namely, we need $\frac{\partial loss}{\partial w}$ and
$\frac{\partial loss}{\partial b}$ under some fixed values of ``x`` and ``y``. To compute those derivatives, we call ``loss.backward()``, and then retrieve the values from ``w.grad`` and ``b.grad``:


**Note:** We can only obtain the ``grad`` properties for the leaf nodes of the computational graph, which have ``requires_grad`` property set to ``True``. For all other nodes in our graph, gradients will not be available.We can only perform gradient calculations using ``backward`` once on a given graph, for performance reasons. If we need to do several ``backward`` calls on the same graph, we need to pass ``retain_graph=True`` to the ``backward`` call.

In [5]:
loss.backward()
print(w.grad)
print(b.grad)
print()

comp_grad = (torch.nn.Sigmoid()(z).detach())/dim_outputs
print(f"Computed gradient is {comp_grad}")

tensor([[0.2992, 0.2116, 0.3322],
        [0.2992, 0.2116, 0.3322],
        [0.2992, 0.2116, 0.3322],
        [0.2992, 0.2116, 0.3322],
        [0.2992, 0.2116, 0.3322]])
tensor([0.2992, 0.2116, 0.3322])

Computed gradient is tensor([0.2992, 0.2116, 0.3322])


# Disabling Gradient Tracking


By default, all tensors with ``requires_grad=True`` are tracking their
computational history and support gradient computation. However, there
are some cases when we do not need to do that, for example, when we have
trained the model and just want to apply it to some input data, i.e. we
only want to do *forward* computations through the network. We can stop
tracking computations by surrounding our computation code with
``torch.no_grad()`` block:


There are other reasons we might want to disable gradient tracking:
  - To mark some parameters in your neural network at **frozen parameters**. This is a very common scenario for [`finetuning a pretrained network`](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)
  - To **speed up computations** when we are only doing forward pass, because computations on tensors that do not track gradients would be more efficient.

In [6]:
z = torch.matmul(x, w)+b
print("Grad present for z:", z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print("Grad present for z:", z.requires_grad)

Grad present for z: True
Grad present for z: False


Another way to achieve the same result is to use the ``detach()`` method
on the tensor:




In [7]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print("Grad present for z:", z.requires_grad)
print("Grad present for z_det:", z_det.requires_grad)

Grad present for z: True
Grad present for z_det: False


# More on Computational Graphs

Conceptually, autograd keeps a record of data (tensors) and all executed operations (along with the resulting new tensors) in a directed acyclic graph (DAG) consisting of [`Function`] (https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function) objects. In this DAG, leaves are the input tensors, roots are the output tensors. By tracing this graph from roots to leaves, we can automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:

- run the requested operation to compute a resulting tensor
- maintain the operation’s *gradient function* in the DAG.

The backward pass kicks off when ``.backward()`` is called on the DAG root. ``autograd`` then:

- computes the gradients from each ``.grad_fn``,
- accumulates them in the respective tensor’s ``.grad`` attribute
- using the chain rule, propagates all the way to the leaf tensors.

**Note** DAGs are dynamic in PyTorch. An important thing to note is that the graph is recreated from scratch; after each ``.backward()`` call, autograd starts populating a new graph. This is exactly what allows you to use control flow statements in your model; we can change the shape, size and operations at every iteration if needed.

# Tensor Gradients and Jacobian Products

In many cases, we have a scalar loss function, and we need to compute the gradient with respect to some parameters. However, there are cases when the output function is an arbitrary tensor. In this case, PyTorch allows us to compute so-called **Jacobian product**, and not the actual gradient.

For a vector function $\vec{y}=f(\vec{x})$, where
$\vec{x}=\langle x_1,\dots,x_n\rangle$ and
$\vec{y}=\langle y_1,\dots,y_m\rangle$, a gradient of
$\vec{y}$ with respect to $\vec{x}$ is given by **Jacobian
matrix**:


\\begin{align}J=\left(\begin{array}{ccc} \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\ \vdots & \ddots & \vdots\\ \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}} \end{array}\right)\end{align}


Instead of computing the Jacobian matrix itself, PyTorch allows you to
compute **Jacobian Product** $v^T\cdot J$ for a given input vector
$v=(v_1 \dots v_m)$. This is achieved by calling ``backward`` with
$v$ as an argument. The size of $v$ should be the same as
the size of the original tensor, with respect to which we want to
compute the product:




In [8]:
inp = torch.eye(5, requires_grad = True)
out = (inp + 1).pow(2) # out = (inp + 1) ^ 2
print(inp) 
print()
print(out)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]], requires_grad=True)

tensor([[4., 1., 1., 1., 1.],
        [1., 4., 1., 1., 1.],
        [1., 1., 4., 1., 1.],
        [1., 1., 1., 4., 1.],
        [1., 1., 1., 1., 4.]], grad_fn=<PowBackward0>)


In [9]:
out.backward(torch.ones_like(inp), retain_graph = True)
print("First call:\n", inp.grad) # Gradient = 2 * (inp + 1)
print()
out.backward(torch.ones_like(inp), retain_graph = True)
print("\nSecond call\n", inp.grad)

First call:
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])


When we call ``backward`` for the second time with the same argument, the value of the gradient is different. This happens because when doing ``backward`` propagation, PyTorch **accumulates the
gradients**, i.e. the value of computed gradients is added to the ``grad`` property of all leaf nodes of computational graph. If we want to compute the proper gradients, we need to zero out the ``grad``
property before. In real-life training an *optimizer* helps us to do this.

In [10]:
inp.grad.zero_()
print("Gradient after zeroing gradients:\n", inp.grad)
print()
out.backward(torch.ones_like(inp), retain_graph=True)
print("Gradient call again after zeroing gradients:\n", inp.grad)

Gradient after zeroing gradients:
 tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

Gradient call again after zeroing gradients:
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])


**Note:** Previously we were calling ``backward()`` function without parameters. This is essentially equivalent to calling ``backward(torch.tensor(1.0))``, which is a useful way to compute the          gradients in case of a scalar-valued function, such as loss during neural network training.

# Further Reading

- [`Autograd Mechanics`](https://pytorch.org/docs/stable/notes/autograd.html)